# 초기 환경 설정

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("ragas_Evaluation_test1")


LangSmith 추적을 시작합니다.
[프로젝트명]
ragas_Evaluation_test1


In [ ]:
# %pip install ragas

In [3]:
%pip install -qU ragas==0.1.16

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-teddynote 0.5.1 requires langchain>=0.3.27, but you have langchain 0.2.17 which is incompatible.
langchain-teddynote 0.5.1 requires langchain-openai>=0.3.30, but you have langchain-openai 0.1.25 which is incompatible.


In [4]:
%pip show ragas

Name: ragas
Version: 0.1.16
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: c:\anaconda3\envs\final_pj\Lib\site-packages
Requires: appdirs, datasets, langchain, langchain-community, langchain-core, langchain-openai, nest-asyncio, numpy, openai, pysbd, tiktoken
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_community.document_loaders import JSONLoader
import json

loader = JSONLoader('./gmp_chunks.jsonl', jq_schema='.', json_lines=True, text_content=False)
docs = loader.load()

In [5]:
import json
docs = []

with open('./gmp_chunks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        docs.append(obj)

In [6]:
docs[:2]

[{'id': '0646aa1d61_annex_15_qualification_and_validation_2-2b20c947f12c-0000',
  'doc_id': '0646aa1d61_annex_15_qualification_and_validation_2-2b20c947f12c',
  'source_path': 'temp_extract/raw/EU/0646aa1d61_annex 15 qualification and validation 2015 final.pdf',
  'title': '0646aa1d61_annex 15 qualification and validation 2015 final',
  'jurisdiction': 'EU',
  'doc_date': '2015',
  'doc_version': None,
  'section_id': None,
  'section_title': None,
  'page_start': 1,
  'page_end': 1,
  'chunk_index': 0,
  'text': 'EU GMP Guide, Annex 15. Qualification and Validation GE010b\n \ngggmmmpppeeeyyyeee \nwww.gmpeye.co.kr \n1 \n \nEUROPEAN COMMISSION \nDIRECTORATE-GENERAL FOR HEALTH AND FOOD SAFETY \n \n2015년 3월 30일, 브뤼셀 \n \n \nEudraLex \n \nVolume 4 \nEU Guidelines for Good Manufacturing Practice \nfor Medicinal Products for Human and Veterinary Use \n \nAnnex 15: Qualification and Validation \n \nLegal basis for publishing the detailed guidelines: Article 47 of Directive 2001/83/EC on \nthe

2. 로드된 문서의 각 metadata에 filename 속성 추가
    - filename속성은 Test Datasets 생성 프로세스에서 활용됨
    - 동일한 문서에 속한 청크를 식별하는 데 사용 됨 (source와 같은 역할)
    - source의 내용을 filename속성에 복사해주기

In [15]:
# 각 Document객체의 metadata에 filename속성 추가
for doc in docs:
    doc.metadata['filename'] = doc.metadata['source']

## 2.2 데이터셋 생성
- Q&A dataset 종류
    - simple: 단순 질의응답 데이터셋
    - reasioning: 추론 능력을 확인할 수 있는 데이터셋
    - multi_context: 여러 문맥을 고려하여 답 생성하는 데이터셋 
    - conditional: 조건부 질의 응답 

1. 모델 생성

In [6]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기 생성 (llm으로 데이터셋/질문 생성)
generator_llm = ChatOpenAI(model='gpt-4o-mini')

# 데이터셋 비평기 생성 (llm으로 질문이 적합한 질문인지 평가)
critic_llm = ChatOpenAI(model='gpt-4o-mini')

# 임베딩 모델 객체 생성
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

c:\anaconda3\envs\final_pj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2. 메모리 객체 생성 준비 (InMemory객체 옵션에 필요한 객체 생성)

In [7]:
# splitter 생성
splitter = RecursiveCharacterTextSplitter(
    chunk_size=550,
    chunk_overlap=150
)

# ragas와 호환되는 llm 생성
# Lanchain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 RAGAS와 호환되도록 함
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model='gpt-4o-mini'))

# 문서에서 주요 구문(KeyPhrase)를 먼저 뽑기 위한 객체 생성(ragas와 호환되는 llm 사용)
# 즉, 문서의 주요 구문에서 질문과 답변을 뽑아냄
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas와 호환되는 embeddings 객체 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore 객체 생성 (key phrase를 메모리에 올리는 역할)
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor
)

3. TestSet 생성해주는 생성기 생성

In [8]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore
)

4. 질의응답 유형별 분포 

In [9]:
# 질문 유형별 분포 설정
# 질문을 10개 생성할 경우 각각 4개, 2개, 2개, 2개씩 생성
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}

In [10]:
from langchain.schema import Document


documents = []
for d in docs:
    # 딕셔너리의 'text' 값을 문서의 page_content로 설정
    page_content = d.get('text', '')
    
    # 'text'를 제외한 나머지는 metadata로 설정
    metadata = {k: v for k, v in d.items() if k != 'text'}
    
    # Document 객체 생성 및 리스트에 추가
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)

# 이제 `page_content` 속성에 정상적으로 접근할 수 있습니다.
print(documents[0].page_content)
# # 출력: 첫 번째 텍스트 덩어리입니다.

print(documents[0].metadata)
# 출력: {'chunk_id': 1, 'source': 'file.jsonl'}

print(documents[0])

EU GMP Guide, Annex 15. Qualification and Validation GE010b
 
gggmmmpppeeeyyyeee 
www.gmpeye.co.kr 
1 
 
EUROPEAN COMMISSION 
DIRECTORATE-GENERAL FOR HEALTH AND FOOD SAFETY 
 
2015년 3월 30일, 브뤼셀 
 
 
EudraLex 
 
Volume 4 
EU Guidelines for Good Manufacturing Practice 
for Medicinal Products for Human and Veterinary Use 
 
Annex 15: Qualification and Validation 
 
Legal basis for publishing the detailed guidelines: Article 47 of Directive 2001/83/EC on 
the Community code relating to medicinal products for human use a n d A r t i c l e 5 1 o f 
Directive 2001/82/EC on the Community code relating to veterinary medicinal products. This 
document provides guidance for the interpretation of the princi ples and guidelines of good 
manufacturing practice (GMP) for medicinal products as laid dow n in Directive 2003/94/EC 
for medicinal products for human use and Directive 91/412/EEC for veterinary use. 
세부 가이드라인 발행의 법적 근거 : 사람 의약품 관련 EC 법률에 관한 디렉티브
{'id': '0646aa1d61_annex_15_qualification_and_

In [31]:
type(documents[0].page_content)

str

In [32]:
type(documents[0].metadata)


dict

In [24]:
%pip install langchain==0.2.16


Note: you may need to restart the kernel to use updated packages.


In [25]:
pip show langchain

Name: langchain
Version: 0.2.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\anaconda3\envs\final_pj\Lib\site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community, langchain-teddynote, ragas
Note: you may need to restart the kernel to use updated packages.


In [11]:
for doc in documents:
    doc.metadata['filename'] = doc.metadata['source_path']

In [12]:
documents[0].metadata

{'id': '0646aa1d61_annex_15_qualification_and_validation_2-2b20c947f12c-0000',
 'doc_id': '0646aa1d61_annex_15_qualification_and_validation_2-2b20c947f12c',
 'source_path': 'temp_extract/raw/EU/0646aa1d61_annex 15 qualification and validation 2015 final.pdf',
 'title': '0646aa1d61_annex 15 qualification and validation 2015 final',
 'jurisdiction': 'EU',
 'doc_date': '2015',
 'doc_version': None,
 'section_id': None,
 'section_title': None,
 'page_start': 1,
 'page_end': 1,
 'chunk_index': 0,
 'filename': 'temp_extract/raw/EU/0646aa1d61_annex 15 qualification and validation 2015 final.pdf'}

In [16]:
documents = documents[:101]

In [17]:
# %pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [18]:
# test dataset 객체 생성
test_dataset = generator.generate_with_langchain_docs(
    documents=documents,
    test_size=10,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False
)

Generating:   0%|          | 0/10 [00:00<?, ?it/s]                [ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 1, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Cleaning verification', 'Chemical analysis', 'Residues of previous product']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['EU GMP Guide', 'Annex 15', 'Ongoing process verification', 'Traditional approach', 'Process validation']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['EU GMP Guide', 'Annex 15', 'Ongoing process verification', 'Traditional approach', 'Process validation']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 2, 'structure': 2, 'relevan

바보야 감자야 잘가

In [19]:
print(type(test_dataset))
print(len(test_dataset.test_data))
test_dataset.test_data[0].__annotations__

<class 'ragas.testset.generator.TestDataset'>
10


{'question': 'str',
 'contexts': 't.List[str]',
 'ground_truth': 't.Union[str, float]',
 'evolution_type': 'str',
 'metadata': 't.List[dict]'}

6. DataFrame으로 변형
    - question: 생성된 질문
    - contexts: 질문의 근거가 되는 청크
    - ground_truth: llm이 만들어낸 정답 답변
    - evolution_type: 질의 유형

In [20]:
# dataframe으로 변형
test_df = test_dataset.to_pandas()
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the significance of analyzing the resi...,[보여 주는 증거 문서를 확립하는 활동이다. \n \n세척 베리피케이션(Cleani...,The answer to given question is not present in...,simple,[{'id': '0646aa1d61_annex_15_qualification_and...,True
1,What regulations govern veterinary use of medi...,[manufacturing practice (GMP) for medicinal pr...,The regulations governing veterinary use of me...,simple,[{'id': '0b26bc7349_eu_gmp_guide_chapter_5_pro...,True
2,What is the significance of the traditional ap...,[historical batch data. \n 과거 배치 데이터와 제조 경험을 통...,The answer to given question is not present in...,simple,[{'id': '0646aa1d61_annex_15_qualification_and...,True
3,What measures should be taken in the context o...,[EU Guide to Good Manufacturing Practice for M...,Checks on yields and reconciliation of quantit...,simple,[{'id': '0b26bc7349_eu_gmp_guide_chapter_5_pro...,True
4,How do GMP guidelines ensure product quality?,"[아니며, 활성 성분 제조업체는 이 부록을 보완적인 가이드라인으로 선택하여 활용할 ...",The answer to given question is not present in...,reasoning,[{'id': '0646aa1d61_annex_15_qualification_and...,True


In [21]:
test_df.shape

(10, 6)

In [22]:
test_df.iloc[0]['ground_truth']

'The answer to given question is not present in context'

7. csv 파일로 저장

In [ ]:
test_df.to_csv('./data/ragas_test_dataset.csv', index=False) # df의 인덱스는 포함 안 함

# 3. 평가

1. csv 파일 로드 

In [ ]:
import pandas as pd

df = pd.read_csv('./data/ragas_test_dataset.csv')
df.head()

2. DataFrame을 Dataset 객체로 변환

In [ ]:
from datasets import Dataset

# ragas에서 평가 시 Dataset클래스로 매핑이 돼있어야 됨
test_dataset = Dataset.from_pandas(df)
test_dataset

3. df의 contexts를 list 타입으로 벼환해주기
- 겉보기엔 list처럼 보이지만 사실 str 값임 

In [ ]:
# contexts 칼럼 타입 확인 
print(type(df.iloc[0]['contexts']))

In [ ]:
import ast

# contexts 칼럼 데이터를 list로 변환 
def conver_to_list(example):
    contexts = ast.literal_eval(example['contexts']) # literal_eval: 문자열을 파이썬 객체로 변환해줌 
    return {'contexts': contexts}

# map
#   입력: Dataset의 각 행이 사용자 정의 함수에 입력으로 들어감
#   출력: 사용자 정의함수가 적용된 새로운 Dataset이 반환됨
test_dataset = test_dataset.map(conver_to_list) 
print(test_dataset)

In [ ]:
# 타입 변환된 거 확인
print(type(test_dataset[0]['contexts']))

4. 예제 프롬프트 파이프라인

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. 문서 로드
loader = PDFPlumberLoader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')
docs = loader.load()[3:-1]

# 2. 문서 분할
splitter = RecursiveCharacterTextSplitter(
    chunk_size=550,
    chunk_overlap=150
)
split_docs = splitter.split_documents(docs)

# 3. 임베딩 객체 생성
embeddings = OpenAIEmbeddings()

# 4. vectorstore 생성 및 저장 
vectorstore = FAISS.from_documents(
    documents=split_docs,
    embedding=embeddings
)

# 5. 검색기 생성
retriever = vectorstore.as_retriever()

# 6. 프롬프트 생성
prompt = PromptTemplate.from_template(
    """
당신은 친절한 챗봇입니다. 사용자의 질문에 대답해주세요.
사용자의 질문이 들어오면 context에 기반하여 question에 대답해주세요.

#Context:
{context}

#Question:
{question}

#Answer:
"""
)

# 7. llm 객체 생성
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

# 8. chain 생성
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

5. batch 데이터셋 생성
    - 다량의 질문을 한 번에 처리할 때 유용

In [ ]:
batch_dataset = [question for question in test_dataset['question']]
batch_dataset[:5]

6. 평가용 질문으로 답변 받아보기 
    - batch 방식 사용

In [ ]:
result = chain.batch(batch_dataset)

# 답변
result

7. 답변을 test_dataset의 answer 컬럼에 저장하기 (answer컬럼 새로 생성해줘야 함)

In [ ]:
# answer 컬럼 덮어 씌우기 또는 추가
if 'answer' in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(['answer']).add_column('answer', result)
else:
    test_dataset = test_dataset.add_column('answer', result)

In [ ]:
print(type(test_dataset))
test_dataset.column_names

## 3.1  답변 평가  

1. Context Recall 
    - 검색된 context가 LLM이 생성한 답변과 얼마나 일치하는지 측정 
        - 10개의 chunks를 뽑아냈는데 10개 모두에 답변에 필요한 내용이 담겨있는 경우 
        - 즉, ground-truth가 10개의 문서에 모두 있는 경우 성능이 좋음을 의미
    - 0~1 사이의 값
    - 값이 높을 수록 좋은 성능을 의미

2. Context Precision
    - 얼마나 관련성 있는 문서가 상위에 배치되었는가?
    - 즉, ground-truth가 포함된 문서가 얼마나 상위에 배치돼있는가를 판단 
    - 예를 들어) 답변에 필요한 문서가 3개인데 3개가 retriever로 뽑은 10개의 문서 중 상위 3위안에 들면 성능은 만점이 나옴

3. Answer Relevancy
    - 생성된 답변이 주어진 prompt에 얼마나 적절한지를 평가 
    - 즉 생성된 답변이 원래 질문의 의도를 얼마나 잘 반영하는지를 측정 
    - 질문의 임베딩과 생성된 답변의 임베딩의 코사인 유사도 측정
        - 답변이 질문과 연관성이 높으면 높은 점수를 받음 
    - 0~1사이의 값을 가지거나 코사인 유사도 특성상 수학적으로 -1~1 사이의 값을 가질 수도 있음 

4. Faithfulness
    - 생성된 답변의 사실적 일관성을 주어진 컨텍스트와 비교하여 측정하는 지표 
    - 즉, 주어진 context에 얼마나 충실히 (정확히) 답변했는지 평가 

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

In [ ]:
print(type(result))

In [ ]:
result_df = result.to_pandas()
result_df.head()

In [ ]:
result_df.loc[:, 'context_precision':'context_recall']
# precision(1.0이 나올 경우 ground-truth가 포함된 context의 유사도가 1순위임을 의미)
# faithfulness(사실 여부 판단)
# answer_relevancy(prompt에따라 잘 답변 했는지 정도)
# context_recall(검색된 context가 LLM이 생성한 답변(answer)과 얼마나 일치하는지)